In [10]:
import re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from glob import glob

In [ ]:
# Low pass filter to filter out unwanted noise from the force plate data.
def low_pass_filter(x, fc = 400, fs = 3500):
    w = fc / (fs / 2)  # Normalize the frequency
    b, a = signal.butter(3, w, "low")

    return signal.filtfilt(b, a, x, axis=0)


In [ ]:
# Read kinetic data from text file and manipulate it into each force plate etc.
#
# az0 = - 0.062;
# a = 0.210
# b = 0.350
#
# The Kistler force plate dimensions are 600 x 900 mm.
#
# Mx = b * (fz1 + fz2 - fz3 - fz4)
# My = a * (-fz1 + fz2 + fz3 - fz4)
# Mz = b * (-fx12 + fx34) + a * (fy14 - fy23)
# Mx' = Mx + Fy*az0
# My' = My - Fx*az0
# cop_x = -My' / Fz
# cop_y = Mx' / Fz
# Tz = Mz - Fy * ax + Fx * ay
# Declare constants used to convert the raw stream of data to metrics.
root_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/grf_kinetic_data/ForcePlateData"
txt_data = sorted(glob(os.path.join(root_dir, "**/*.txt"), recursive=True))
num_force_plates = 8
num_data_columns = 8
az0 = -0.062
a = 0.210
b = 0.350
for txt_fname in txt_data:
    out_fname = txt_fname.split("/")[-1][:-4]
    with open(txt_fname, "r") as f:
        data_str = f.read()
        data = re.split("[\t\n]", data_str)
        data = np.array(data[:-1], dtype=np.float32)
        # TODO: There are meant to be 8 force plates but we only have data for 7 of them?
        try:
            force_plate_data = data.reshape((-1, num_force_plates, num_data_columns))
        except:
            f.close()
            print(f"Not correct number of force plates (skip): {out_fname}")
            continue
        # Run through each force plate and determine values
        df_list = []
        for i in range(num_force_plates):
            force_plate_data_filtered = low_pass_filter(force_plate_data[:, i], fc=400)
            fx12 = force_plate_data_filtered[:, 0]
            fx34 = force_plate_data_filtered[:, 1]
            fy14 = force_plate_data_filtered[:, 2]
            fy23 = force_plate_data_filtered[:, 3]
            fz1 = force_plate_data_filtered[:, 4]
            fz2 = force_plate_data_filtered[:, 5]
            fz3 = force_plate_data_filtered[:, 6]
            fz4 = force_plate_data_filtered[:, 7]
            Fx = fx12 + fx34
            Fy = fy14 + fy23
            Fz = fz1 + fz2 + fz3 + fz4
            Mx = b * (fz1 + fz2 - fz3 - fz4)
            My = a * (-fz1 + fz2 + fz3 - fz4)
            Mz = b * (-fx12 + fx34) + a * (fy14 - fy23)
            Mx_ = Mx + Fy*az0
            My_ = My - Fx*az0
            cop_x = -My_ / Fz
            cop_y = Mx_ / Fz
            Tz = Mz - Fy * cop_x + Fx * cop_y
            df_list.append(pd.DataFrame(np.array([Fx, Fy, Fz, cop_x, cop_y, Tz]).T,
                                columns=["Fx", "Fy", "Fz", "cop_x", "cop_y", "Tz"]))
        # Convert data to a single coordinate frame.
        ref_plate = df_list[0].copy()
        Fx = ref_plate["Fx"].values
        Fy = ref_plate["Fy"].values
        Fz = ref_plate["Fz"].values
        dax = 0.6 * np.array([float(i) for i in range(num_force_plates)])
        day = [0.0] * num_force_plates
        Mx = (day[0] + ref_plate["cop_y"].values) * Fz
        My = -(dax[0] + ref_plate["cop_x"].values) * Fz
        Mz = ((dax[0] + ref_plate["cop_x"].values) * Fy - (day[0] + ref_plate["cop_y"].values) * Fx)
        for i in range(1, num_force_plates):
            fp_data = df_list[i]
            Fx += fp_data["Fx"].values
            Fy += fp_data["Fy"].values
            Fz += fp_data["Fz"].values
            Mx += (day[i] + fp_data["cop_y"].values) * fp_data["Fz"].values
            My -= (dax[i] + fp_data["cop_x"].values) * fp_data["Fz"].values
            Mz += ((dax[i] + fp_data["cop_x"].values) * fp_data["Fy"].values -
                (day[i] + fp_data["cop_y"].values) * fp_data["Fx"].values)
        Mx_ = Mx
        My_ = My
        for i in range(num_force_plates):
            fp_data = df_list[i]
            Mx_ += az0 * fp_data["Fy"].values
            My_ -= az0 * fp_data["Fx"].values
        cop_x = -My_ / Fz
        cop_y = Mx_ / Fz
        Tz = Mz - Fy*cop_x + Fx*cop_y
        df_list.append(pd.DataFrame(np.array([Fx, Fy, Fz, cop_x, cop_y, Tz]).T,
                            columns=["Fx", "Fy", "Fz", "cop_x", "cop_y", "Tz"]))
        # Concatenate results and write to file.
        df = pd.concat(df_list, keys=[i for i in range(num_force_plates + 1)], axis=0)
        df.index.set_names(["force_plate", "frame"], inplace=True)
        try:
            df.to_csv(
                os.path.join(
                    root_dir,
                    f"{out_fname}.csv"))
            df.to_hdf(os.path.join(
                root_dir,
                f"{out_fname}.h5"),
                    "force_plate_data_df",
                    format="table",
                    mode="w")
        except:
            print(f"Failed to save results: {out_fname}")
            continue


In [ ]:
# Plotting results for global coordinate frame.
# The Kistler force plate dimensions are 600 x 900 mm.
root_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/grf_kinetic_data/ForcePlateData"
df_data = sorted(glob(os.path.join(root_dir, "**/*.h5"), recursive=True))
x_axis_m = num_force_plates * 0.6
y_axis_m = 0.9
for df_file in df_data:
    df = pd.read_hdf(df_file)
    out_fname = df_file.split("/")[-1][:-3]
    # Plot each force plate separately.
    for i in range(num_force_plates):
        fp_data = df.query(f"force_plate == {i}")
        fig = plt.figure(figsize=(16, 12), dpi=120)
        ax = fig.add_subplot(111)
        ax.plot(fp_data["Fx"].values, color="b", label="Fx")
        ax.plot(fp_data["Fy"].values, color="g", label="Fy")
        ax.plot(fp_data["Fz"].values, color="r", label="Fz")
        ax.set_title("GRF")
        ax.set_xlabel("Frames")
        ax.set_ylabel("Force (N)")
        ax.legend()
        plt.savefig(os.path.join(root_dir, f"{out_fname}_fp{i+1}.png"))
        plt.close()
    # Plot data referenced to a single world location.
    values = df.query(f"force_plate == {num_force_plates}")
    fig = plt.figure(figsize=(16, 12), dpi=120)
    axs = fig.subplots(1, 2).flatten()
    axs[0].plot(values["Fx"].values, color="b", label="Fx")
    axs[0].plot(values["Fy"].values, color="g", label="Fy")
    axs[0].plot(values["Fz"].values, color="r", label="Fz")
    axs[0].set_title("GRF")
    axs[0].set_xlabel("Frames")
    axs[0].set_ylabel("Force (N)")
    axs[0].legend()
    axs[1].scatter(values["cop_x"].values, values["cop_y"].values)
    axs[1].set_xlim((-0.6, x_axis_m))
    axs[1].set_ylim((-y_axis_m/2, y_axis_m/2))
    axs[1].set_aspect('equal', 'box')
    axs[1].set_title("COP")
    axs[1].set_xlabel("X")
    axs[1].set_ylabel("Y")
    plt.savefig(os.path.join(root_dir, f"{out_fname}.png"))
    plt.close()
    fig = plt.figure(figsize=(16, 12), dpi=120)
    axs = fig.subplots(2, 1).flatten()
    axs[0].plot(values["cop_x"].values)
    axs[1].plot(values["cop_y"].values)
    axs[0].set_title("COP X")
    axs[1].set_title("COP Y")
    axs[0].set_xlabel("Frames")
    axs[1].set_xlabel("Frames")
    plt.savefig(os.path.join(root_dir, f"{out_fname}_cop.png"))
    plt.close()